## Imports

In [ ]:
import sys

import joblib
from sklearn import metrics
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from tqdm.auto import tqdm

sys.path.append('..')
from utils import cv_kfold, train_validate_split

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Data Loading

In [ ]:
def read_file(filename: str) -> pd.DataFrame:
    return pd.DataFrame([
        (l.split()[0], l.split()[1], ' '.join(l.split()[3:]))
        for l in open(filename)
        ], columns=['class', 'sent', 'text']
    )

In [ ]:
df_train = read_file('../datasets/train.txt')
df_test = read_file('../datasets/test.txt')

len(df_train), len(df_test)

In [ ]:
X = df_train['text'].values
y = df_train['class'].values
y_sent = df_train['sent'].values

## Experiments

In [ ]:
vectorizers = {
    'Count': CountVectorizer(),
    'Tfidf': TfidfVectorizer(),
    'Hashing': HashingVectorizer(),
}
models = {
    'LogReg': LogisticRegression(),
    'SVM': SVC(),
    'LinearSVM': LinearSVC(),
    'RandomForest': RandomForestClassifier(),
    'ExtraTrees': ExtraTreesClassifier(),
    'GradientBoosting': GradientBoostingClassifier(),
    'NaiveBayes': MultinomialNB(),
    'KNeighbors': KNeighborsClassifier(),
}

In [ ]:
df_scores = pd.DataFrame(columns=vectorizers.keys(), index=models.keys())
df_time = pd.DataFrame(columns=vectorizers.keys(), index=models.keys())
scorer = lambda *x: metrics.f1_score(*x, average='micro')

joblib_memory = joblib.Memory()
for model_name, model in tqdm(models.items(), total=len(models), desc='models'):
    for vec_name, vec in tqdm(vectorizers.items(), total=len(vectorizers), desc='vectorizer'):
        pipeline = Pipeline(
            steps=[
                ('vec', vec),
                ('cls', model)
            ],
            memory=joblib_memory,
        )

        try:
            kfold_result = cv_kfold(pipeline, X, y, scorer=scorer, k=5)
            df_scores.loc[model_name, vec_name] = kfold_result['oof_score']
            df_time.loc[model_name, vec_name] = kfold_result['mean_time']

            # X_train, X_val, y_train, y_val = train_test_split(X, y)
            # validate_results = train_validate_split(pipeline, X_train, y_train, X_val, y_val, scorer, verbose=0)
            # df_scores.loc[model_name, vec_name] = validate_results['score']
            # df_time.loc[model_name, vec_name] = validate_results['time']
        except Exception as e:
            df_scores.loc[model_name, vec_name] = None
            df_time.loc[model_name, vec_name] = None

df_scores

In [ ]:
df_time